In [4]:
import json
import pandas as pd

In [7]:
data = []

with open('News_Category_Dataset_v2.json') as f:
    for line in f:
        json_dict = json.loads(line)
        current_line = [json_dict['category'], 
                        json_dict['headline'], 
                        json_dict['short_description'], 
                        json_dict['link'],
                        json_dict['date']]
        data.append(current_line)

print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

